In [1]:
import pandas as pd

from wsi.utils import raw_data_path, processed_data_path
from wsi.shocks.population import build_population_df


In [2]:
file_path = raw_data_path("shocks", 'public_emdat_custom_request.xlsx')
emdat = pd.read_excel(file_path, sheet_name="EM-DAT Data")

cols = ['ISO', 'Country', 'Subregion', 'Disaster Subgroup', 'Disaster Type', 'Latitude', 'Longitude','Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'Total Affected']

#emdat = emdat[emdat['ISO'].isin(iso_codes)][cols]
emdat = emdat[emdat["End Year"] >= 1995]
emdat['Total Deaths'] = emdat['Total Deaths'].fillna(0)
emdat = emdat[emdat['Total Deaths'] > 0]

emdat[cols]

,ISO,Country,Subregion,Disaster Subgroup,Disaster Type,Latitude,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected
2,SOM,Somalia,Sub-Saharan Africa,Climatological,Drought,NaN,NaN,2000,1.0,NaN,2001,NaN,NaN,21.0,1200000.0
3,AGO,Angola,Sub-Saharan Africa,Hydrological,Flood,NaN,NaN,2000,1.0,8.0,2000,1.0,15.0,31.0,70000.0
4,BGD,Bangladesh,Southern Asia,Meteorological,Extreme temperature,NaN,NaN,2000,1.0,NaN,2000,1.0,NaN,49.0,NaN
6,IRN,Iran (Islamic Republic of),Southern Asia,Meteorological,Storm,NaN,NaN,2000,1.0,18.0,2000,1.0,18.0,3.0,5500.0
7,MOZ,Mozambique,Sub-Saharan Africa,Hydrological,Flood,NaN,NaN,2000,1.0,26.0,2000,3.0,27.0,800.0,4500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,ESP,Spain,Southern Europe,Meteorological,Storm,NaN,NaN,2025,1.0,27.0,2025,1.0,28.0,1.0,NaN
10300,MYS,Malaysia,South-eastern Asia,Hydrological,Flood,NaN,NaN,2025,1.0,28.0,2025,1.0,30.0,5.0,12500.0
10301,BRA,Brazil,Latin America and the Caribbean,Meteorological,Storm,NaN,NaN,2025,2.0,5.0,2025,2.0,7.0,9.0,130.0
10302,AUS,Australia,Australia and New Zealand,Hydrological,Flood,NaN,NaN,2025,2.0,1.0,2025,2.0,6.0,2.0,6000.0


In [3]:
# aggregate by year and sum deaths/affected
emdat_agg = emdat.groupby(['ISO', 'End Year'], as_index=False)[['Total Deaths', 'Total Affected']].sum().rename(columns={'ISO':'ISO_code', 'End Year': 'Year'})

all_years = list(range(1995, 2025))
all_iso_codes = emdat_agg['ISO_code'].unique()
full_index = pd.MultiIndex.from_product([all_iso_codes, all_years],
                                          names=['ISO_code', 'Year'])
emdat_agg = emdat_agg.set_index(['ISO_code', 'Year'])
emdat_agg = emdat_agg.reindex(full_index, fill_value=0).reset_index()

emdat_agg

,ISO_code,Year,Total Deaths,Total Affected
0,AFG,1995,0.0,0.0
1,AFG,1996,0.0,0.0
2,AFG,1997,0.0,0.0
3,AFG,1998,0.0,0.0
4,AFG,1999,0.0,0.0
...,...,...,...,...
6205,ZWE,2020,0.0,0.0
6206,ZWE,2021,3.0,2400.0
6207,ZWE,2022,750.0,6551.0
6208,ZWE,2023,2.0,0.0


In [4]:
# Calculate percentage of population for Total Deaths and Total Affected
df_pop = build_population_df()
merged_df = emdat_agg.merge(df_pop, left_on=['ISO_code', 'Year'], right_on=['ISO_code', 'Year'], how='left')
merged_df['Total Deaths (%)'] = (merged_df['Total Deaths'] / merged_df['Population']) * 100
merged_df['Total Affected (%)'] = (merged_df['Total Affected'] / merged_df['Population']) * 100

merged_df.head()


,ISO_code,Year,Total Deaths,Total Affected,Population,Total Deaths (%),Total Affected (%)
0,AFG,1995,0.0,0.0,16712942.0,0.0,0.0
1,AFG,1996,0.0,0.0,17418729.0,0.0,0.0
2,AFG,1997,0.0,0.0,18107803.0,0.0,0.0
3,AFG,1998,0.0,0.0,18796380.0,0.0,0.0
4,AFG,1999,0.0,0.0,19523611.0,0.0,0.0


In [5]:
merged_df.to_csv(processed_data_path('shocks','death_disasters.csv'))